## Local Model Testing to Prompt Engineer our Movie Reccomender
Here we are loading in our models locally to try and optimize our prompt engineering

Now We'll load our models in.

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

print("Getting pipelines:")
liquid = pipeline("text-generation", model="LiquidAI/LFM2.5-1.2B-Instruct",return_full_text=False)
Qwen = pipeline("text-generation", model="Qwen/Qwen2.5-0.5B-Instruct",return_full_text=False)
Geilim = pipeline("text-generation", model="NoesisLab/Geilim-1B-Instruct",trust_remote_code=True,return_full_text=False)
# Llama = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

c:\Users\caleb\Dropbox\Documents\WPI\25-26\C Term\MLOps\movie-mood-recomender\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Getting pipelines:


Device set to use cpu
Device set to use cpu
Device set to use cpu


Now it's time to set up our prompts. We are using langchain + Pydantic to do this.

In [6]:
from pydantic import BaseModel, Field
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

# Define the JSON schema we want model outputs to conform to
class MovieRecommendation(BaseModel):
    """A movie recommendation based on mood."""
    title: str = Field(description="The title of the recommended movie")
    genre: str = Field(description="The genre of the movie")
    reason: str = Field(description="Brief explanation of why this movie matches the mood")

# Wrap a transformer pipeline with LangChain
# IMPORTANT: return_full_text=False so only the generated text is returned, not the prompt
Qwen.return_full_text = False
llm = HuggingFacePipeline(pipeline=Qwen)

# Parser that validates model output against the Pydantic schema
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

# Simpler prompt that small models can follow more reliably
prompt = PromptTemplate(
    template=(
        "Recommend a movie based on the user's query. "
        "Respond with ONLY a JSON object with these keys: "
        "\"title\" (movie title), \"genre\" (movie genre), \"reason\" (why it fits).\n\n"
        "Query: {query}\n\n"
        "JSON:"
    ),
    input_variables=["query"],
)

# Chain: prompt -> LLM -> parse & validate JSON
chain = prompt | llm | parser

result = chain.invoke({"query": "Recommend a movie for someone feeling adventurous"})
print(result)

title='The Great Gatsby' genre='Fiction' reason='The Great Gatsby is a classic novel about love, loss, and ambition that has inspired countless filmmakers to tell its story.'
